In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn import mixture
%matplotlib inline

In [2]:
def outlier_detection(df, ft):
    df.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [3]:
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR 
    upper_bound = Q3 + 1.5 * IQR
    filt = (lower_bound>df[ft]) | (upper_bound<df[ft])
    ls = df[filt].index
    return ls

In [4]:
def remove(df, ls):
    ls = list(set(ls))
    df_clean = df.drop(index=ls)
    return df_clean 

In [29]:
def calculate_classification_data(df, split_time, is_pca_on, pca_n_components, cluster_n):
    df_clean = data_cleaning(df).copy()
    if is_pca_on:
        #pca transform
        pca_data = df_clean.copy()
        pca_data_standard = scale(pca_data)
        pca_model = PCA(n_components=pca_n_components, random_state=2021)
        pca_model.fit(pca_data_standard)
        for i in range(1,pca_n_components+1):
            pca_data[f'pca_{i}'] = pca_model.transform(pca_data_standard)[:,i-1]
        data_processed = pca_data.loc[:, 'pca_1':f'pca_{pca_n_components}'].copy()
    else:
        data_processed = pd.DataFrame(scale(df_clean), index=df_clean.index, columns=df_clean.columns).copy()
    #clustering
    df_train = data_processed[(data_processed.index>pd.to_datetime(split_time)-pd.DateOffset(years=7)) & (data_processed.index<pd.to_datetime(split_time))].copy()
    df_test = data_processed[(pd.to_datetime(split_time)<=data_processed.index) & (data_processed.index<=pd.to_datetime(split_time)+pd.DateOffset(years=1))].copy()
        
    km = KMeans(n_clusters=cluster_n, random_state=2021)
    results = km.fit_predict(df_train)
    #calculate the Silhouetter Score
    score = metrics.silhouette_score(df_train, km.labels_, metric='euclidean')
    print(f'Silhouette Score: {score}')
    df_train['label'] = results
#     df_train['label'] = df_train['label'].shift(-1)
    df_train.dropna(inplace=True)
    
    df_train.reset_index(drop=True)
    df_train.reset_index(drop=True)
    
    df_train.to_csv(f'train_{split_time[0:4]}_no_pca_no_shift.csv')
    df_test.to_csv(f'test_{split_time[0:4]}_no_pca_no_shift.csv')
    return df_train, df_test
    
        

In [30]:
def data_cleaning(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df = df.apply(lambda x: x.pct_change())
    df.dropna(inplace=True)
    outlier_list = []
    for col in df.columns:
        outlier_list.extend(outliers(df, col))
    df_clean = remove(df, outlier_list)
    return df_clean

In [31]:
df = pd.read_csv('Data/rawData_Feb11.csv')
df

,Date,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM
0,1990-01-02,200.241,17.24,NaN,0.11,NaN,NaN,NaN,NaN,NaN,1.2146,NaN,NaN
1,1990-01-03,200.116,18.19,NaN,0.10,NaN,NaN,NaN,NaN,NaN,1.2096,NaN,NaN
2,1990-01-04,198.646,19.22,NaN,0.14,NaN,NaN,NaN,NaN,NaN,1.2287,NaN,NaN
3,1990-01-05,196.850,20.11,NaN,0.20,NaN,NaN,NaN,NaN,NaN,1.2358,NaN,NaN
4,1990-01-08,197.525,20.26,NaN,0.23,NaN,NaN,NaN,NaN,NaN,1.2452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8374,2022-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.970,48.56
8375,2022-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.330,49.03
8376,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.350,49.80
8377,2022-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.630,49.47


In [18]:
train_2018_no_pca_no_shift, test_2018_no_pca_no_shift = calculate_classification_data(df, '2018-01-01', False, 8, 2)
train_2019_no_pca_no_shift, test_2019_no_pca_no_shift = calculate_classification_data(df, '2019-01-01', False, 8, 2)
train_2020_no_pca_no_shift, test_2020_no_pca_no_shift = calculate_classification_data(df, '2020-01-01', False, 8, 2)
train_2021_no_pca_no_shift, test_2021_no_pca_no_shift = calculate_classification_data(df, '2021-01-01', False, 8, 2)

Silhouette Score: 0.19091367279812835
Silhouette Score: 0.18940361917035275
Silhouette Score: 0.17958249395692993
Silhouette Score: 0.17761300426185292


In [32]:
train_2018_no_pca_no_shift, test_2018_no_pca_no_shift = calculate_classification_data(df, '2018-01-01', False, 8, 2)

Silhouette Score: 0.19091367279812835


In [60]:
train_2018['y'] = train_2018['label'].shift(1)
train_2018.dropna(inplace=True)

In [12]:
train_2018_no_pca_no_shift.reset_index(drop=True)

,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,label
0,1.040954,-0.354611,0.424050,0.724424,1.947947,1.772630,-0.756082,0.223098,1.546021,1.438929,1.439790,1.026191,1
1,2.587361,-1.674205,2.273316,0.711376,-1.022856,1.800195,0.606812,1.410556,0.185336,2.332677,2.892800,2.476652,1
2,-0.658071,-0.230967,-0.441719,0.356039,0.006766,-0.097411,-0.885268,-0.594027,-0.121931,-0.319801,-0.852462,-0.892856,0
3,0.239252,-0.723597,0.519774,1.008676,-1.102913,0.676213,-0.441930,0.566159,-1.038919,-0.934373,-0.216051,-0.128284,1
4,0.885408,0.516463,0.417167,-0.094500,-1.148895,-0.127283,-0.091878,0.721881,1.109721,0.024565,0.256351,0.043212,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,0.154666,-0.076114,-0.203377,0.698798,0.236401,0.634621,0.118686,-0.354115,-2.514806,0.049629,-0.781074,0.596605,0
1290,-0.301856,0.647526,-0.372203,0.686665,0.062870,0.542900,0.900068,1.012473,0.742665,-0.250637,0.293151,0.799610,1
1291,-0.100126,0.507137,-0.203268,-0.747645,-0.401093,-0.007950,0.306806,0.689975,0.290011,0.590876,0.116903,0.174921,1
1292,0.121063,-0.395925,-0.058845,2.342641,0.155102,1.426621,1.120235,0.848896,0.547185,1.089187,0.116462,0.630748,1


In [ ]:
train_2018_no_pca_no_shift.groupby('label').agg(['mean', 'std']).transpose()

In [28]:
train_2019_no_pca_no_shift.groupby('label').agg(['mean', 'std']).transpose()

label                 0         1
RAY      mean -0.710110  0.616550
         std   0.666278  0.708270
VIX      mean  0.646385 -0.571512
         std   0.847098  0.760891
QQQ      mean -0.625184  0.549200
         std   0.757752  0.769057
T10Y3M   mean -0.200498  0.213523
         std   0.911040  0.889559
CL1      mean -0.352005  0.263496
         std   0.958465  0.910573
HG1      mean -0.351713  0.264235
         std   0.878178  0.956869
GC1      mean -0.167562  0.067073
         std   0.997511  0.991580
XAG      mean -0.305520  0.176099
         std   0.924383  0.933576
VNQ      mean -0.402442  0.318354
         std   0.893149  0.883418
EURtoUSD mean -0.228826  0.205477
         std   0.971937  1.032141
VXUS     mean -0.763225  0.684090
         std   0.680214  0.666837
EEM      mean -0.711112  0.661196
         std   0.745753  0.716695

In [37]:
train_2019.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.101101  0.004082
      std   2.132772  2.045445
pca_2 mean -0.026515 -0.035622
      std   1.411292  1.437818
pca_3 mean  0.006772  0.004501
      std   1.017987  1.074369
pca_4 mean  0.023457  0.045789
      std   0.941696  0.951505
pca_5 mean -0.022429  0.027400
      std   0.888139  0.832009
pca_6 mean  0.009709  0.021880
      std   0.817773  0.804553
pca_7 mean  0.001243  0.014802
      std   0.727812  0.758405
pca_8 mean -0.042481  0.011679
      std   0.694154  0.658690

In [38]:
train_2020.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.089637  0.002619
      std   2.117335  2.030744
pca_2 mean -0.020301 -0.037553
      std   1.404899  1.426434
pca_3 mean  0.003569  0.020551
      std   1.039725  1.088490
pca_4 mean  0.005504  0.040554
      std   0.936952  0.949386
pca_5 mean -0.019440  0.025084
      std   0.883818  0.833288
pca_6 mean  0.007907  0.014933
      std   0.821588  0.814081
pca_7 mean -0.001835  0.014721
      std   0.742515  0.768959
pca_8 mean -0.039503  0.007276
      std   0.693687  0.655838

In [39]:
train_2021.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.068618 -0.048819
      std   2.143774  2.027236
pca_2 mean  0.026032 -0.049989
      std   1.397406  1.426464
pca_3 mean -0.000414  0.011116
      std   1.087306  1.095950
pca_4 mean -0.009574  0.044486
      std   0.939768  0.942816
pca_5 mean -0.015625  0.013196
      std   0.893176  0.857122
pca_6 mean -0.005993  0.013042
      std   0.838490  0.815781
pca_7 mean  0.019051  0.012016
      std   0.798136  0.782206
pca_8 mean -0.025330  0.009359
      std   0.715721  0.663529

#### cluster the whole dataset

In [23]:
df_clean

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,-0.001461,0.034115,-0.003645,-0.001406,0.000658,-0.007399,-0.006024,-0.008492,-0.013152,0.000779,0.004489,-0.005764,-0.002993
2022-01-11,0.009908,-0.051031,0.011454,0.019105,0.015022,0.014274,-0.006061,0.038221,0.017693,0.010952,0.013957,0.001540,0.003620
2022-01-14,0.000467,-0.055145,0.000407,-0.014383,0.006223,-0.000935,0.044304,0.020701,-0.027607,-0.002690,-0.005457,-0.009052,-0.003841


In [29]:
km = KMeans(n_clusters=3, random_state=2021)
results_whole = km.fit_predict(df_clean)
#calculate the Silhouetter Score
score = metrics.silhouette_score(df_clean, km.labels_, metric='euclidean')
print(f'Silhouette Score: {score}')

Silhouette Score: 0.2496869919356694


In [30]:
df_test_whole = df_clean.copy()
df_test_whole['label'] = results_whole-1

In [31]:
df_test_whole.to_csv('benchmark_data_3_cluster.csv')

In [32]:
df_test_whole.head()

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,label
Date,,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416,-1
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871,0
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888,0
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316,1
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160,0


#### find expected return and vol for each group

In [33]:

df_test_whole.groupby('label').agg(['mean', 'std']).transpose()

label                -1         0         1
RAY      mean -0.004872  0.001254  0.007005
         std   0.004299  0.003702  0.004813
VIX      mean  0.068080 -0.005807 -0.074200
         std   0.029603  0.019636  0.028418
IWV      mean -0.004897  0.001255  0.006989
         std   0.004318  0.003732  0.004838
EEM      mean -0.002378  0.001042  0.003122
         std   0.009774  0.009660  0.009642
QQQ      mean -0.005135  0.001982  0.007815
         std   0.006355  0.005831  0.006958
VXUS     mean -0.004633  0.000944  0.006007
         std   0.005290  0.005255  0.005615
T10Y3M   mean -0.007835 -0.002782  0.010282
         std   0.034408  0.031933  0.032450
CL1      mean -0.004897  0.001225  0.008320
         std   0.017619  0.017053  0.019203
HG1      mean -0.001845  0.000894  0.003025
         std   0.010279  0.010700  0.011709
GC1      mean  0.000615  0.000410 -0.000072
         std   0.006620  0.006271  0.006245
XAG      mean -0.001106  0.000764  0.001798
         std   0.011231  0.010267  0.010898
VNQ      mean -0.002816  0.000587  0.004860
         std   0.008259  0.007522  0.008222
EURtoUSD mean -0.000048  0.000113  0.000053
         std   0.003795  0.004146  0.004302

In [ ]:
#based on the above results, 1: bull market, -1: bear market 0: stable market

In [39]:
#Hierarchical Clustering
hr_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hr_cluster.fit_predict(df_clean)
hr_score = metrics.silhouette_score(df_clean, hr_cluster.labels_, metric='euclidean')
print(f'Silhouette Score: {hr_score}')

Silhouette Score: 0.22456252552673217


In [66]:
pd.to_datetime('2019-01-01')-pd.DateOffset(years=7)

Timestamp('2012-01-01 00:00:00')

In [20]:
df_predict = pd.read_csv('2018prediction_no_pca_no_shift.csv')

In [22]:
df_predict = df_predict.drop()

,label_1_PREDICTION,label_0_PREDICTION,label_PREDICTION,THRESHOLD,POSITIVE_CLASS,DEPLOYMENT_APPROVAL_STATUS,Date,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM
0,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-02,1.142000,-2.000259,2.110663,0.389975,-0.107476,-0.667111,0.728120,1.251256,-0.590684,1.058672,1.282395,1.908592
1,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-03,0.741047,-1.052835,1.071970,0.386282,1.255274,-0.614989,0.244393,-0.312143,-0.486197,-0.877660,0.850476,0.919048
2,1.000000,6.000000e-10,1,0.5,1,APPROVED,2018-01-04,0.377592,0.253400,0.012428,0.752042,0.331728,0.119851,0.320121,0.427529,-2.283447,1.038000,1.038949,0.426772
3,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-05,0.837449,0.112870,1.115370,1.100315,-0.632793,-0.986268,0.058669,-0.071715,-0.059976,-0.778539,0.811417,0.817093
4,0.000352,9.996484e-01,0,0.5,1,APPROVED,2018-01-08,0.081992,0.710568,0.297180,-1.395840,0.240929,-0.193938,-0.223605,-0.483644,0.528276,-1.235481,-0.204870,-0.099708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.000000,1.000000e+00,0,0.5,1,APPROVED,2018-12-10,-0.065662,-0.353676,1.172441,-0.832192,-1.977434,-1.344945,-0.374244,-0.602696,-0.931740,-0.490848,-1.350592,-1.219502
180,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-12-12,0.756320,-0.140382,0.950246,0.013280,-0.663471,0.047259,0.271590,0.976839,-2.058693,1.081663,2.082874,1.638536
181,0.230823,7.691773e-01,0,0.5,1,APPROVED,2018-12-13,-0.565669,-0.580471,-0.179974,0.013280,1.700277,-0.236886,-0.213447,0.039307,-0.886924,-0.177683,-0.333639,-0.126020
182,0.000000,1.000000e+00,0,0.5,1,APPROVED,2018-12-20,-2.861108,2.123578,-2.148039,3.098110,-1.811857,-0.699311,1.300972,0.961556,-2.005932,1.452072,-0.310920,0.729030


In [ ]:
df_predict.groupby('label_PREDICTION').agg(['mean', 'std']).transpose()